# Preprocessing Merged Weather + Fire Data
#### Fire Atlas Dataset + NCDC GSOM Data
Andrew McDonald // CSE 847 // April 2021

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
cmap = plt.get_cmap("tab10")

In [2]:
d_high = gpd.read_file("data/low_n_high_d.shp")
d_med = gpd.read_file("data/med_n_med_d.shp")
d_low = gpd.read_file("data/high_n_low_d.shp")
with pd.option_context('display.max_columns', None):  # more options can be specified also
    display(d_high)

,fire_ID,latitude_x,longitude_,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,expansion,fire_line,speed,direction,direction_,landcover,landcover_,tile_ID,pop_est,continent,name_x,iso_a3,gdp_md_est,nearest_st,elevation,mindate,maxdate,latitude_y,name_y,datacovera,id,elevationU,longitud_1,distance_f,month,date,station,DP01,DP10,DP1X,DYXP,EMXP,PRCP,CDSD,CLDD,DT00,DT32,DX32,DX70,DX90,DYNT,DYXT,EMNT,EMXT,HDSD,HTDD,TAVG,TMAX,TMIN,DSNW,DYSN,EMSN,SNOW,DSND,DYSD,EMSD,geometry
0,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,3.73,6.20,2.68,8,northwest,7,Open shrublands,h03v06,326625791,North America,United States of America,USA,18560000.0,32923,1228.3,2013-01-01,2017-03-01,20.684200,"KAUPO GAP HAWAII, HI US",1.0000,GHCND:USR0000HKAU,METERS,-156.151900,0.110925,2,2016-02-01T00:00:00,GHCND:USR0000HKAU,NaN,NaN,NaN,NaN,NaN,NaN,0.4,0.1,0.0,0.0,0.0,15.0,0.0,20160208.0,20160227.0,7.2,23.9,205.7,72.6,15.8,20.7,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-156.24786 20.62712)
1,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,1.29,2.47,1.39,4,southeast,7,Open shrublands,h03v07,326625791,North America,United States of America,USA,18560000.0,32941,709.0,2003-05-01,2021-03-01,19.795000,"PUU WAAWAA HAWAII, HI US",0.9487,GHCND:USR0000HPUW,METERS,-155.845300,0.050968,3,2016-03-01T00:00:00,GHCND:USR0000HPUW,NaN,NaN,NaN,NaN,NaN,NaN,48.9,23.1,0.0,0.0,0.0,31.0,0.0,20160327.0,20160328.0,13.3,27.8,17.1,0.9,19.1,23.7,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-155.89512 19.80629)
2,6,42.2438,-121.8750,0.86,5.56,2016-08-09,222,2016-08-09,222,1,0.86,1.85,0.79,0,none,12,Croplands,h08v04,326625791,North America,United States of America,USA,18560000.0,21880,1255.5,2013-11-01,2021-03-01,42.228588,"KLAMATH FALLS 0.5 NNW, OR US",1.0000,GHCND:US1ORKL0016,METERS,-121.774503,0.101642,8,2016-08-01T00:00:00,GHCND:US1ORKL0016,1.0,0.0,0.0,20160806.0,0.3,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20160831.0,0.0,0.0,NaN,NaN,NaN,POINT (-121.87496 42.24378)
3,7,42.2313,-121.8730,0.64,4.63,2016-08-16,229,2016-08-18,231,3,0.21,0.46,0.49,4,southeast,12,Croplands,h08v04,326625791,North America,United States of America,USA,18560000.0,21880,1255.5,2013-11-01,2021-03-01,42.228588,"KLAMATH FALLS 0.5 NNW, OR US",1.0000,GHCND:US1ORKL0016,METERS,-121.774503,0.098534,8,2016-08-01T00:00:00,GHCND:US1ORKL0016,1.0,0.0,0.0,20160806.0,0.3,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20160831.0,0.0,0.0,NaN,NaN,NaN,POINT (-121.87333 42.23128)
4,8,41.8938,-122.9790,20.37,40.74,2016-08-28,241,2016-09-12,256,16,1.27,2.66,1.49,8,northwest,1,Evergreen Needleleaf forest,h08v04,326625791,North America,United States of America,USA,18560000.0,32609,1674.3,1991-08-01,2021-03-01,41.775000,"COLLINS BALDY CALIFORNIA, CA US",0.9775,GHCND:USR0000CCOL,METERS,-122.950300,0.122218,8,2016-08-01T00:00:00,GHCND:USR0000CCOL,NaN,NaN,NaN,NaN,NaN,NaN,235.5,119.7,0.0,0.0,0.0,31.0,3.0,20160809.0,20160819.0,8.3,35.0,54.3,10.8,21.9,27.5,16.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.97924 41.89378)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9190,56789,40.8521,-78.1691,0.43,2.78,2016-04-14,105,2016-04-14,105,1,0.43,0.93,0.46,0,none,4,Deciduous Broadleaf forest,h12v04,326625791,North America,United States of America,USA,18560000.0,30357,524.3,1998-04-01,2021-03-01,40.872910,"PHILIPSBURG 2 S, PA US",0.9927,GHCND:USC00366921,METERS,-78.216150,0.051447,4,2016-04-01T00:00:00,GHCND:USC00366921,15.0,7.0,0.0,20160429.0,15.0,61.7,0.0,0.0,0.0,13.0,0.0,7.0,0.0,20160410.0,20160419.0,-7.2,26.1,2976.4,294.0,8.5,15.4,1.6,3.0,20160409.0,30.0,89.0,3.0,20160409.0,25.0,POINT (-78.16909 40.85211)
9191,56790,40.4188,-76.7056,0.64,3.70,2015-11-14,318,2015-11-14,318,1,0.64,1.39,0.64,0,none,5,Mixed forest,h12v04,326625791,North America,Un

In [3]:
d_high.columns

Index(['fire_ID', 'latitude_x', 'longitude_', 'size', 'perimeter',
       'start_date', 'start_DOY', 'end_date', 'end_DOY', 'duration',
       'expansion', 'fire_line', 'speed', 'direction', 'direction_',
       'landcover', 'landcover_', 'tile_ID', 'pop_est', 'continent', 'name_x',
       'iso_a3', 'gdp_md_est', 'nearest_st', 'elevation', 'mindate', 'maxdate',
       'latitude_y', 'name_y', 'datacovera', 'id', 'elevationU', 'longitud_1',
       'distance_f', 'month', 'date', 'station', 'DP01', 'DP10', 'DP1X',
       'DYXP', 'EMXP', 'PRCP', 'CDSD', 'CLDD', 'DT00', 'DT32', 'DX32', 'DX70',
       'DX90', 'DYNT', 'DYXT', 'EMNT', 'EMXT', 'HDSD', 'HTDD', 'TAVG', 'TMAX',
       'TMIN', 'DSNW', 'DYSN', 'EMSN', 'SNOW', 'DSND', 'DYSD', 'EMSD',
       'geometry'],
      dtype='object')

In [4]:
exclusions = ['fire_ID',        # exclude as irrelevant index information
                'start_date',   # exclude as duplicate of start_DOY
                'end_date',     # exclude as duplicate of end_DOY
                'duration',     # exclude as duplicate of start/end DOY
                'direction',    # exclude as duplicate of direction_string
                'landcover',    # exclude as duplicate of landcover_string
                'tile_ID',      # exclude as irrelevant MODIS information
                'pop_est',      # exclude as irrelevant (all US data)
                'continent',    # exclude as irrelevant (all US data)
                'name_x',       # exclude as irrelevant (all US data)
                'iso_a3',       # exclude as irrelevant (all US data)
                'gdp_md_est',   # exclude as irrelevant (all US data)
                'nearest_st',   # exclude as irrelevant weather station info
                'mindate',      # exclude as irrelevant weather station info
                'maxdate',      # exclude as irrelevant weather station info
                'latitude_y',   # exclude as duplicate of latitude from first DF
                'longitud_1',   # exclude as duplicate of longitude from first DF
                'name_y',       # exclude as duplicate of end_DOY
                'datacovera',   # exclude as irrelevant weather station info
                'id',           # exclude as irrelevant weather station info
                'elevationU',   # exclude as irrelevant weather station info
                'month',        # exclude as duplicate of start/end DOY
                'date',         # exclude as duplicate of start/end DOY
                'station',      # exclude as irrelevant weather station info
                'geometry',     # exclude as duplicate of lat/lon from first DF
]
X_high = d_high.drop(exclusions, axis=1).dropna()
X_med = d_med.drop(exclusions, axis=1).dropna()
X_low = d_low.drop(exclusions, axis=1).dropna()
print(X_high.shape, X_med.shape, X_low.shape)

(1414, 42) (4298, 33) (7672, 19)


In [5]:
targets = ['size', 'perimeter']
X_high, y_high = X_high.drop(targets, axis=1), X_high[targets]
X_med, y_med = X_med.drop(targets, axis=1), X_med[targets]
X_low, y_low = X_low.drop(targets, axis=1), X_low[targets]
print(X_high.shape, X_med.shape, X_low.shape)
print(y_high.shape, y_med.shape, y_low.shape)

(1414, 40) (4298, 31) (7672, 17)
(1414, 2) (4298, 2) (7672, 2)


In [6]:
for X in [X_high, X_med, X_low]:

    # convert date columns to day-of-year integer
    for feature in ["DYNT", "DYSD", "DYSN", "DYXP", "DYXT"]:
        if feature in X.columns:
            X[feature] = X[feature].astype(int).astype(str).str[4:]
            X[feature] = 30 * X[feature].str[:2].astype(int) + ("0" + X[feature].str[2:]).astype(int)

In [7]:
X_high.to_csv("data/X_H.csv", index=False)
y_high.to_csv("data/y_H.csv", index=False)

X_med.to_csv("data/X_M.csv", index=False)
y_med.to_csv("data/y_M.csv", index=False)

X_low.to_csv("data/X_L.csv", index=False)
y_low.to_csv("data/y_L.csv", index=False)
